# Webserv

## External Functs

### - execve()

La función execve() es una función del sistema operativo Unix que se utiliza en C++ para ejecutar otro programa desde un programa en ejecución.

La sintaxis de execve() es la siguiente:

__int execve(const char *filename, char *const argv[], char *const envp[]);__


- filename: es una cadena de caracteres que especifica el nombre del archivo ejecutable que se desea ejecutar.
- argv: es un puntero a un arreglo de cadenas de caracteres que representan los argumentos para el nuevo programa. El primer elemento de este arreglo es el nombre del programa.
- envp: es un puntero a un arreglo de cadenas de caracteres que representan las variables de entorno para el nuevo programa.

El valor devuelto por execve() es -1 si ocurre un error y, de lo contrario, no se devuelve nada.

In [ ]:
#include <unistd.h>

int main()
{
    char const *argv[] = { "/bin/ls", "-l", NULL };
    char const *envp[] = { NULL };

    execve(argv[0], argv, envp);

    /* Si se ejecuta esta línea, significa que la llamada a execve() ha fallado. */
    perror("execve");
    return 1;
}

En este ejemplo, el programa llama a execve() para ejecutar el comando "ls -l" del sistema operativo. El primer elemento del arreglo argv es "/bin/ls", que es la ruta completa del comando "ls". El segundo elemento es "-l", que es el argumento para listar el contenido de un directorio en formato largo. El tercer elemento es NULL, lo que indica el final de la lista de argumentos.

El arreglo envp se establece en NULL porque no se necesitan variables de entorno para este ejemplo.

Si la llamada a execve() tiene éxito, el programa actual se reemplaza por el comando "ls -l". Si la llamada falla, se muestra un mensaje de error.

## - dup, dup2

Tanto la función dup() como la función dup2() duplican un descriptor de archivo. La principal diferencia entre ellas es que con dup2() se puede especificar el número de descriptor de archivo que se desea utilizar como copia, mientras que con dup() no. 

Por ejemplo, si se desea redirigir la salida estándar a un archivo, se puede utilizar dup() para duplicar el descriptor de archivo de la salida estándar, y dup2() para redirigir la salida estándar al archivo especificado. Al finalizar, se debe utilizar dup2() nuevamente para restaurar el descriptor de archivo original de la salida estándar.

Supongamos que queremos redirigir la salida estándar a un archivo llamado "output.txt". Para ello, podemos utilizar las funciones dup() y dup2() de la siguiente manera:


In [ ]:
#include <unistd.h>
#include <fcntl.h>
#include <iostream>

int main() {
    // Abrimos el archivo output.txt
    int fd = open("output.txt", O_CREAT|O_WRONLY, 0644);
  
    // Duplicamos el descriptor de archivo de la salida estándar
    int stdout_copy = dup(STDOUT_FILENO);
  
    // Redirigimos la salida estándar al archivo output.txt
    dup2(fd, STDOUT_FILENO);
  
    // Escribimos en la salida estándar
    std::cout << "Hola, mundo!\n";
  
    // Restauramos el descriptor de archivo original de la salida estándar
    dup2(stdout_copy, STDOUT_FILENO);
  
    // Cerramos el archivo output.txt y la copia del descriptor de archivo de la salida estándar
    close(fd);
    close(stdout_copy);
  
    return 0;
}


##### En este ejemplo:
1. primero abrimos el archivo "output.txt" utilizando la función open(). 
2. Luego, utilizamos la función dup() para duplicar el descriptor de archivo de la salida estándar y guardamos la copia en la variable "stdout_copy". 
3. Después, utilizamos la función dup2() para redirigir la salida estándar al archivo "output.txt". 
4. Escribimos en la salida estándar, y finalmente, utilizamos dup2() nuevamente para restaurar el descriptor de archivo original de la salida estándar. 
5. Cerramos el archivo y la copia del descriptor de archivo de la salida estándar.

## - std::strerror

### strerror() prototype

__char* strerror( int errnum );__

El strerror()toma un número como argumento de error que es un valor entero que representa el código de error. Esta función convierte el código de error en una cadena adecuada que describe el error. [Más información](https://www.programiz.com/cpp-programming/library-function/cstring/strerror)

In [ ]:
#include <cstring>
#include <cmath>
#include <cstdio>
#include <cerrno>
#include <iostream>

int main()
{
    float log_neg = log(-2.5);
    std::cout << "Log of negative number : " << std::strerror(errno) << "\n";

    /* example.txt does not exist */
    FILE * fp = fopen("example.txt","r");
    if (fp == NULL)
        std::cout << "Error opening file : " << std::strerror(errno) << "\n";

    return 0;
}

## - gai_strerror()

La función gai_strerror() se utiliza para obtener una descripción legible del error devuelto por una función de resolución de nombres, como getaddrinfo() o getnameinfo().

Toma como argumento el código de error devuelto por la función de resolución de nombres y devuelve una cadena de caracteres con una descripción legible del error.
[man gai_strerror()](https://man7.org/linux/man-pages/man3/gai_strerror.3p.html)

In [ ]:
#include <iostream>
#include <netdb.h>
#include <string.h>

int main() {
    struct addrinfo hints, *res;
    int status;

    memset(&hints, 0, sizeof hints);
    hints.ai_family = AF_UNSPEC; 
    hints.ai_socktype = SOCK_STREAM; 

    if ((status = getaddrinfo("www.google.com", "http", &hints, &res)) != 0) {
        std::cerr << "Error en getaddrinfo: " << gai_strerror(status) << std::endl;
        return 1;
    }

    return 0;
}

In [ ]:
std::cerr << "Error en getaddrinfo: " << gai_strerror(status) << std::endl;

En este ejemplo, utilizamos la función getaddrinfo() para obtener información de dirección para la dirección "www.google.com" y el servicio "http". Si getaddrinfo() devuelve un error, utilizamos la función gai_strerror() para obtener una descripción  del error y la imprimimos en la salida de error estándar utilizando std::cerr.

La función gai_strerror() toma como argumento el código de error devuelto por la función de resolución de nombres (en este caso, el valor de la variable status), y devuelve una cadena de caracteres con una descripción legible por humanos del error.

Por ejemplo, si el error es "Name or service not known" (nombre o servicio desconocido), la salida será:

__Error en getaddrinfo: Name or service not known__


## htons(), htonl(), ntohs() y ntohl()

Las funciones htons(), htonl(), ntohs() y ntohl() son utilizadas en redes de ordenadores para convertir datos de un formato de orden de bytes a otro. El orden de bytes se refiere a la forma en que los bytes de datos se organizan en un bloque de memoria.

La función htons() convierte un número entero de formato de orden de bytes del sistema anfitrión a un formato de red (big-endian), mientras que la función htonl() hace lo mismo para un número entero largo.

Por otro lado, la función ntohs() convierte un número entero de formato de red al formato del sistema anfitrión, mientras que la función ntohl() hace lo mismo para un número entero largo.

Un ejemplo de uso sería convertir una dirección IP de formato de orden de bytes del sistema anfitrión a un formato de red para enviarla a través de una red, y luego convertirla de nuevo al formato del sistema anfitrión para utilizarla en la aplicación.

In [ ]:
#include <iostream>
#include <arpa/inet.h>

int main() {
    uint32_t ip_addr = inet_addr("192.168.1.1"); // dirección IP en formato de host

    uint32_t network_ip_addr = htonl(ip_addr); // convertir dirección IP a formato de red

    uint32_t host_ip_addr = ntohl(network_ip_addr); // convertir dirección IP de nuevo al formato de host

    std::cout << "Dirección IP: " << inet_ntoa(*(struct in_addr*)&ip_addr) << std::endl;
    std::cout << "Dirección IP en formato de red: " << inet_ntoa(*(struct in_addr*)&network_ip_addr) << std::endl;
    std::cout << "Dirección IP en formato de host: " << inet_ntoa(*(struct in_addr*)&host_ip_addr) << std::endl;

    return 0;
}


En este ejemplo, primero utilizamos la función inet_addr() para convertir una cadena que representa una dirección IP en formato de host a un número entero de 32 bits.

Luego, utilizamos la función htonl() para convertir la dirección IP a formato de red y la función ntohl() para convertirla de nuevo al formato de host.

Por último, imprimimos la dirección IP en los tres formatos utilizando la función inet_ntoa(), que convierte un número entero de 32 bits a una cadena que representa una dirección IP en formato de host. [Más información](https://www.gta.ufrj.br/ensino/eel878/sockets/htonsman.html) [Información extra](https://topic.alibabacloud.com/a/ntohs-ntohl-htonshtonl-comparison-and-detailed-explanation_8_8_20182862.html)